In [15]:
import pandas as pd

# Load the CSV files into DataFrames
csv1 = pd.read_csv('/content/indonesiagelap_labelled_Daffa.csv', delimiter=';')  # Replace with your file path (has id, full_text, sentiment)
csv2 = pd.read_csv('/content/indonesiagelap_labelled_fahmi.csv', delimiter=';')  # Replace with your file path (has id, full_text, sentiment)
csv3 = pd.read_csv('/content/indonesiagelap_unlabelled_Radam.csv', delimiter=';')  # Replace with your file path (has full_text, sentiment)

print(csv1.columns)
print(csv2.columns)
print(csv3.columns)
# Remove the 'id' column from csv1 and csv2 as instructed
csv2_id = csv2['id']  # Store the 'id' column
csv2 = csv2.drop(columns=['id'])  # Remove the 'id' column
# csv3 = csv3.drop(columns=['id'])  # Remove the 'id' column # This line is throwing the error!

# Instead of dropping, since it is a string, we can split the column into 3 using ',' as a delimiter
csv3 = csv3['id,full_text,sentiment'].str.split(',', expand=True)
# Assign column names
csv3.columns = ['id', 'full_text', 'sentiment']

# Now you can drop the 'id' column
csv3 = csv3.drop(columns=['id'])


# Assuming the first column is 'full_text' and the second column is 'sentiment'
sentiment_columns = ['sentiment']

# Create a new list to store the final sentiment values
final_sentiments = []

# Iterate through the rows of the CSVs
for i in range(len(csv1)):
    sentiments = [csv1.iloc[i]['sentiment'], csv2.iloc[i]['sentiment'], csv3.iloc[i]['sentiment']]

    # Count the frequency of each sentiment value
    sentiment_counts = {'negatif': sentiments.count('negatif'), 'netral': sentiments.count('netral'), 'positif': sentiments.count('positif')}

    # Get the sentiment with the highest vote or '?' if there's a tie
    sorted_counts = sorted(sentiment_counts.items(), key=lambda x: x[1], reverse=True)

    if sorted_counts[0][1] > sorted_counts[1][1]:
        final_sentiments.append(sorted_counts[0][0])  # Take the one with the most votes
    else:
        final_sentiments.append('?')  # If there's a tie

# Create a new DataFrame with the results
final_df = pd.DataFrame({
    'id': csv2_id,  # Use the stored 'id' column
    'full_text': csv1['full_text'],  # Assuming all CSVs have the same text
    'final_sentiment': final_sentiments
})

# Save the result to a new CSV file
final_df.to_csv('most_voted_sentiment-v1.csv', index=False)

print("Final CSV created successfully!")

Index(['full_text', 'sentiment'], dtype='object')
Index(['id', 'full_text', 'sentiment'], dtype='object')
Index(['id,full_text,sentiment'], dtype='object')
Final CSV created successfully!


In [16]:
# Load the final CSV file
final_df = pd.read_csv('/content/most_voted_sentiment-v1.csv')

# Analyze the distribution of sentiments in the final output
sentiment_analysis = final_df['final_sentiment'].value_counts()

print("Sentiment Distribution in Final Output:")
print(sentiment_analysis)

# Optionally, you can analyze if there were any '?' values (indicating a tie)
tie_count = sentiment_analysis.get('?', 0)
print(f"\nNumber of Tied Rows ('?'): {tie_count}")


Sentiment Distribution in Final Output:
final_sentiment
negatif    1606
netral      281
?           158
positif     112
Name: count, dtype: int64

Number of Tied Rows ('?'): 158
